# Сеточный мир

## Постановка задачи

Условия: задана матрица состояний 5х5, функция ценности перехода из одного состояния в другое, $\gamma = 0.9$. Стратегия выбора перехода (действия) - равновероятная.

Задача: для каждого состояния вычислить его ценность. Вычисления провести двумя методами: путём решения СЛАУ и методом Монте-Карло.

<img src="mesh_world_problem.png" width=600 height=600/>

## Решение

### Монте-Карло

Определим функцию, которая делает броски по Монте-Карло и для перехода из одного состояния в другое пользуется предоставленной стратегией.

Стратегия - это функция, которая принимает на вход текущее состояние и возвращает следующее состояние и награду за переход в следующее состояние.

In [63]:
from typing import Callable, Tuple

import numpy as np

def mc_solution(
        strategy: Callable[[int, int], Tuple[int, int, float]],
        width: int=5, 
        height: int=5, 
        gamma: float=0.9, 
        exp_cnt: int=10**6
    ):
    '''
    Monte-Carlo based solution. Model `exp_cnt` experiments, extract average out of results.
    Arguments:
    * width, height - size of statuses' matrix.
    * gamma - decay coefficient.
    * exp_cnt - number of experiments per state.
    * strategy - callable object, which accepts current state (starting from (1,1)) and returns next state (also starting from (1,1))
and reward for passing to this state.
    
    Returns numpy.ndarray of size height*width, which elements are value function of corresponding state.
    '''
    def _make_throw(i, j, gamma, gamma_i = -1):
        eps = 1/10**10
        result_i = 0.
        gamma_i = 1. if gamma_i == -1 else gamma_i
        
        next_i, next_j, reward = strategy(i, j)
        result_i += reward
        if gamma_i >= eps:
            result_i += gamma_i * _make_throw(next_i, next_j, gamma, gamma_i*gamma)
        
        return result_i
    
    result = np.zeros((height, width))
    
    for i in range(height):
        for j in range(width):
            total_value = 0.
            
            for _ in range(exp_cnt):
                total_value += _make_throw(i+1, j+1, gamma)
            result[i,j] = total_value / exp_cnt
            
    return result

Определим стратегию, которая соответствует нашей задаче.

In [61]:
import numpy as np

def strategy(i: int, j: int):
    possible_states = [
        (i+1, j),
        (i-1, j),
        (i, j+1),
        (i, j-1)
    ]
    reward = 0.
    
    next_state = possible_states[np.random.randint(0, len(possible_states))]
    
    # Check for special states.
    if i == 1 and j == 2:
        next_state = (5,2)
        reward = 10.
    elif i == 1 and j == 4:
        next_state = (3,4)
        reward = 5.
        
    # Check if we are out of grid.
    if next_state[0] < 1 or next_state[0] > 5 or \
       next_state[1] < 1 or next_state[1] > 5:
        next_state = (i,j)
        reward = -1.
    
    return *next_state, reward

In [67]:
print(mc_solution(strategy, exp_cnt=100))

[[ 3.67577553  9.0201432   5.16116386  4.879547    0.8786607 ]
 [ 0.67416033  2.85436687  2.33963723  1.71869365  0.33817851]
 [-0.22989749  0.78508371  0.47520011  0.26074648 -0.51542042]
 [-0.92593223 -0.4721308  -0.11833887 -0.3299982  -0.92748807]
 [-1.67378479 -1.14624043 -0.90443456 -1.11698606 -1.45284828]]


In [68]:
print(mc_solution(strategy, exp_cnt=1000))

[[ 3.18998517  9.12378622  4.8068365   5.10643625  1.23618533]
 [ 1.14162823  2.90670846  2.12347752  1.73394727  0.26376396]
 [-0.25214037  0.56125874  0.46788667  0.11057135 -0.52175786]
 [-0.98357589 -0.38727904 -0.29676681 -0.47804009 -0.96894075]
 [-1.59937897 -1.0472233  -0.92292897 -1.14800085 -1.59396707]]


In [65]:
print(mc_solution(strategy, exp_cnt=10000))

[[ 3.38559175  9.10508137  4.76808648  5.16468559  1.24074107]
 [ 1.21789897  3.06073118  2.12267474  1.80069478  0.24472429]
 [-0.20123055  0.59199218  0.48739771  0.23498679 -0.55325175]
 [-0.94229253 -0.35285149 -0.26201975 -0.44296111 -1.01985765]
 [-1.5993659  -1.07901301 -0.95246321 -1.09656551 -1.60469628]]
